### Full Evaluating CPM with CEBaB

This note book is based on `cebab_evaluation.ipynb`. It takes all the saved results data from running that script, and aggregate portable results. This notebook is only for sorting saved results. Nothing more.


In [1]:
import pickle
import pandas as pd

In [2]:
with open('../proxy_training_results/BERT-results/results.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)